In [1]:
from elasticsearch import Elasticsearch

In [2]:
# Create the client instance
client = Elasticsearch(
    "http://localhost:9200"
)

# Successful response!
client.info()

ObjectApiResponse({'name': 'LAPTOP-Q7SNUCG2', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'yhlAun-SQlmUfzFTFaRwIg', 'version': {'number': '8.3.2', 'build_type': 'zip', 'build_hash': '8b0b1f23fbebecc3c88e4464319dea8989f374fd', 'build_date': '2022-07-06T15:15:15.901688194Z', 'build_snapshot': False, 'lucene_version': '9.2.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [18]:
client.indices.create(
    index="test-chinese", 
    settings={
        'analysis': {
            'analyzer': {
                # we must set the default analyzer
                "default": {
                    "type": "smartcn"
                }
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test-chinese'})

In [17]:
client.indices.delete(index="test-chinese")

ObjectApiResponse({'acknowledged': True})

In [19]:
client.indices.analyze(index="test-chinese", text="中国人民大学")

ObjectApiResponse({'tokens': [{'token': '中国', 'start_offset': 0, 'end_offset': 2, 'type': 'word', 'position': 0}, {'token': '人民', 'start_offset': 2, 'end_offset': 4, 'type': 'word', 'position': 1}, {'token': '大学', 'start_offset': 4, 'end_offset': 6, 'type': 'word', 'position': 2}]})

In [17]:
client.search(
    index="test-english", 
    query={
        "combined_fields": {
            "query": "one",
            "fields": ["title", "abstract"],
        }
    },
    highlight={
        "fields": {
            "title": {
                "pre_tags" : ["<em class='text-primary'>"],
                "post_tags": ["</em>"]
            },
            "abstract": {
                "pre_tags" : ["<em class='text-primary'>"],
                "post_tags": ["</em>"]
            }
        }
    }
)

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 2.7395916, 'hits': [{'_index': 'test-english', '_id': '3-yg8IEBv82c98Tz6i_f', '_score': 2.7395916, '_source': {'title': 'this is the One title', 'abstract': 'this is the One abstract', 'body': 'this is the One body'}, 'highlight': {'abstract': ["this is the <em class='text-primary'>One</em> abstract"], 'title': ["this is the <em class='text-primary'>One</em> title"]}}]}})

In [5]:
from elasticsearch.helpers import bulk
def gendata():
    num2word = {0: 'Zero', 1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five', \
            6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine', 10: 'Ten', \
            11: 'Eleven', 12: 'Twelve', 13: 'Thirteen', 14: 'Fourteen', \
            15: 'Fifteen', 16: 'Sixteen', 17: 'Seventeen', 18: 'Eighteen', 19: 'Nineteen'}

    for i in range(10):
        yield {
            "_index": "test-english",
            "title": f"this is the {num2word[i]} title",
            "abstract": f"this is the {num2word[i]} abstract",
            "body": f"this is the {num2word[i]} body"
        }

bulk(client, gendata())

(10, [])

In [20]:
from elasticsearch.helpers import bulk
def gendata():
    num2word = {0: '零', 1: '一', 2: '二', 3: '三', 4: '四', 5: '五', \
            6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine', 10: 'Ten', \
            11: 'Eleven', 12: 'Twelve', 13: 'Thirteen', 14: 'Fourteen', \
            15: 'Fifteen', 16: 'Sixteen', 17: 'Seventeen', 18: 'Eighteen', 19: 'Nineteen'}

    for i in range(6):
        yield {
            "_index": "test-chinese",
            "title": f"这是第{num2word[i]}篇文章的标题",
            "abstract": f"这是第{num2word[i]}篇文章的摘要",
            "body": f"这是第{num2word[i]}篇文章的正文"
        }

bulk(client, gendata())

(6, [])